In [8]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

In [9]:
# ----------------------------
# Configuration
# ----------------------------
CSV_FILE_PATH = "netflix_titles.csv"
TABLE_NAME = "netflix_raw"

SERVER = r"DESKTOP-844MAL1\MSSQLSERVER"
DATABASE = "master"
DRIVER = "ODBC Driver 17 for SQL Server"


In [10]:
# ----------------------------
# Create Connection String
# ----------------------------
connection_string = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"Trusted_Connection=yes;"
)

params = urllib.parse.quote_plus(connection_string)

engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={params}",
    fast_executemany=True
)


In [ ]:
# ----------------------------
# Load Data into SQL Server
# ----------------------------
try:
    with engine.begin() as connection:
        df = pd.read_csv(CSV_FILE_PATH)
        df.to_sql(
            TABLE_NAME,
            con=connection,
            index=False,
            if_exists="append",
            method="multi"
        )
    print("✅ Data successfully loaded into SQL Server.")

except Exception as e:
    print("❌ Error while loading data:", e)

In [ ]:
# ----------------------------
# Data Validation / Exploration
# ----------------------------
print("\nPreview Data:")
print(df.head())

print("\nRecord with show_id = 's5023':")
print(df[df["show_id"] == "s5023"])

print("\nMax description length:")
print(df["description"].dropna().str.len().max())

print("\nMissing values per column:")
print(df.isna().sum())